In [13]:
import random
import math
from tqdm import tqdm_notebook

In [14]:
#A few helper functions representing a common activation function and its derivative
def sigmoid(x):
    return 1/(1 + math.exp(-x))

def sigmoid_derivative(s):
    return s * (1.0 - s)

def squared_error(output, target):
    return (output - target) ** 2

def squared_error_derivative(output, target):
    return 2 * (output - target)

In [15]:
#Neuron to be instaniated in NeuralNetwork
class Neuron(object):
    def __init__(self, input_count):
        self.bias = random.random()
        self.weights = [random.random() for i in range(input_count)]
        self.input_count = input_count
    
    #Takes the weighted sum of inputs then applies a activation function to normalize the sum
    def getOutput(self, inputs):
        #TODO: assert(len(inputs) == self.input_count)
        weighted_input_sum = self.bias + sum([(self.weights[i] * inputs[i]) for i in range(len(self.weights))])
        self.output = sigmoid(weighted_input_sum)
        return self.output

In [16]:
class NeuralNetwork(object):
    def __init__(self, layer_size_ls):
        #Create layers with inputs the size of the previous layers output count
        self.layers = []
        for i in range(1, len(layer_size_ls)):
            layer_size = layer_size_ls[i]
            prev_layer_size = layer_size_ls[i-1]
            
            #Each layer is represented by a list of neurons of a size with inputs large enough for the previous layer
            self.layers.append([Neuron(prev_layer_size) for i in range(layer_size)]) 
    
    def predict(self, inputs):
        layer_output = inputs
        for layer in self.layers:
            layer_output = [neuron.getOutput(layer_output) for neuron in layer]
        return layer_output

In [21]:
class BackpropagationNNTrainer(object):
    def __init__(self, learning_rate=0.2, activation_func=sigmoid, activation_func_derivative=sigmoid_derivative, error_func=squared_error, error_func_derivative=squared_error_derivative):
        self.learning_rate = learning_rate
        self.activation_func = activation_func
        self.activation_func_derivative = activation_func_derivative
        self.error_func = error_func
        self.error_func_derivative = error_func_derivative
    
    #Feed forward training algorithm
    def back_propagate(self, network, target):
        #1. Compute errors and deltas for the output layer
        output_layer = network.layers[-1]
        for i in range(len(output_layer)):
            neuron = output_layer[i]
            neuron.error = self.error_func_derivative(neuron.output, target[i])
            neuron.delta = neuron.error * self.activation_func_derivative(neuron.output)

        #2. Apply derivative of sigmoid to next layer from each layer (output layer error in the case of last hidden layer)  ... dot-product of weights of each neuron by delta of outputs???
        for i in reversed(range(len(network.layers) - 1)):
            layer = network.layers[i]
            next_layer = network.layers[i+1]

            for neuron_i in range(len(layer)):
                neuron = layer[neuron_i]
                neuron.error = 0.0
                for next_neuron in next_layer:
                    neuron.error += next_neuron.delta * next_neuron.weights[neuron_i]
                neuron.delta = neuron.error * self.activation_func_derivative(neuron.output)
        #Revered layers, output -> input
            #For each neuron on each layer, go to the next layer get weight from each neuron on that layer and the delta associated

    def update_weights(self, network, inputs, learning_rate):
        for i in range(len(network.layers)):
            curr_layer = network.layers[i]

            #outputs of previous layer are the input of this layer
            if i != 0:
                inputs = [neuron.output for neuron in network.layers[i-1]]

            for neuron in curr_layer:
                scaled_delta = learning_rate * neuron.delta
                for j in range(len(inputs)):
                    neuron.weights[j] -= scaled_delta * inputs[j]
                neuron.bias -= scaled_delta
                
    def train(self, network, rows, learning_rate=0.2, n_epoch=1):
        #start_time = time.time()
        for epoch in tqdm_notebook(range(n_epoch)):
            #sum_error = 0.0
            for row in rows:
                X = row[:-1]
                y = row[-1]

                output = network.predict(X)
                #sum_error += abs(sum(self.computeError(target)))
                self.back_propagate(network, y)
                self.update_weights(network, X, learning_rate) #Check that this should be inputs passed in
            #print(">epoch %d: %.3f" % (epoch, sum_error))

In [22]:
#Build an AND dataset
dataset = []
for i in range(1000):
    vals = [random.randint(0,1), random.randint(0,1)]
    dataset.append([vals[0], vals[1], [vals[0] and vals[1]]])

n = NeuralNetwork([2, 5, 1])
trainer = BackpropagationNNTrainer()
trainer.train(n, dataset, n_epoch=100)

#Test that the network represents an AND gate
assert(round(n.predict([0,0])[0]) == 0)
assert(round(n.predict([1,0])[0]) == 0)
assert(round(n.predict([0,1])[0]) == 0)
assert(round(n.predict([1,1])[0]) == 1)

In [23]:
#Build an XOR gate dataset, with 4 hidden layer!!!
dataset = []
for i in range(1000):
    vals = [random.randint(0,1), random.randint(0,1)]
    dataset.append([vals[0], vals[1], [(not (vals[0] and vals[1])) and (vals[0] or vals[1])]])

n = NeuralNetwork([2, 5, 5, 1])
trainer = BackpropagationNNTrainer(learning_rate=0.1)
trainer.train(n, dataset, n_epoch=200)

#Test that the network represents an XOR gate
assert(round(n.predict([0,0])[0]) == 0)
assert(round(n.predict([1,0])[0]) == 1)
assert(round(n.predict([0,1])[0]) == 1)
assert(round(n.predict([1,1])[0]) == 0)

In [24]:
#Build an AND dataset
dataset = []
for i in range(1000):
    vals = [random.randint(0,1), random.randint(0,1), random.randint(0,1), random.randint(0,1)]
    dataset.append([vals[0], vals[1], vals[2], vals[3], [vals[0] and vals[1] and vals[2] and vals[3]]])

n = NeuralNetwork([4, 5, 1])
trainer = BackpropagationNNTrainer(learning_rate=0.1)
trainer.train(n, dataset, n_epoch=200)

#Test that the network represents an AND gate
assert(round(n.predict([0,0,0,0])[0]) == 0)
assert(round(n.predict([0,0,0,1])[0]) == 0)
assert(round(n.predict([0,0,1,0])[0]) == 0)
assert(round(n.predict([0,0,1,1])[0]) == 0)
assert(round(n.predict([0,1,0,0])[0]) == 0)
assert(round(n.predict([0,1,0,1])[0]) == 0)
assert(round(n.predict([0,1,1,0])[0]) == 0)
assert(round(n.predict([0,1,1,1])[0]) == 0)
assert(round(n.predict([1,0,0,0])[0]) == 0)
assert(round(n.predict([1,0,0,1])[0]) == 0)
assert(round(n.predict([1,0,1,0])[0]) == 0)
assert(round(n.predict([1,0,1,1])[0]) == 0)
assert(round(n.predict([1,1,0,0])[0]) == 0)
assert(round(n.predict([1,1,0,1])[0]) == 0)
assert(round(n.predict([1,1,1,0])[0]) == 0)
assert(round(n.predict([1,1,1,1])[0]) == 1)

In [25]:
#Build an SumToOne dataset
dataset = []
for i in range(1000):
    vals = [random.random(), random.random(), ]
    dataset.append([vals[0], vals[1], [(vals[0]+vals[1]) >= 1.0]])

n = NeuralNetwork([2, 10, 1])
trainer = BackpropagationNNTrainer(learning_rate=0.1)
trainer.train(n, dataset, n_epoch=200)

#Test that the network represents a SumToOne gate
print("Should be lower than 0.5")
print(n.predict([0.3, 0.4]))
print(n.predict([0.1, 0.2]))
print(n.predict([0.45, 0.45]))
print(n.predict([0.49, 0.49]))

print("")

print("Should be higher than 0.5")
print(n.predict([0.3, 1.0]))
print(n.predict([0.6, 0.5]))
print(n.predict([0.50, 0.50]))
#it's so close


Should be lower than 0.5
[9.140601020797631e-09]
[3.011579357456661e-11]
[0.00010415661262241913]
[0.04348022886371581]

Should be higher than 0.5
[0.9999999593997994]
[0.9986217579183516]
[0.19174162154598118]


In [26]:
#if it's classification maybe I should just round things to bind them into groups???

In [ ]:
#Build an AND dataset
dataset = []
for i in range(1000):
    vals = [random.randint(0,1), random.randint(0,1), random.randint(0,1), random.randint(0,1)]
    dataset.append([vals[0], vals[1], vals[2], vals[3], [vals[0] and vals[1] and vals[2] and vals[3]]])

n = NeuralNetwork([4, 5, 5, 1])
trainer = BackpropagationNNTrainer(learning_rate=0.1)
trainer.train(n, dataset, n_epoch=500)

#Test that the network represents an AND gate
assert(round(n.predict([0,0,0,0])[0]) == 0)
assert(round(n.predict([0,0,0,1])[0]) == 0)
assert(round(n.predict([0,0,1,0])[0]) == 0)
assert(round(n.predict([0,0,1,1])[0]) == 0)
assert(round(n.predict([0,1,0,0])[0]) == 0)
assert(round(n.predict([0,1,0,1])[0]) == 0)
assert(round(n.predict([0,1,1,0])[0]) == 0)
assert(round(n.predict([0,1,1,1])[0]) == 0)
assert(round(n.predict([1,0,0,0])[0]) == 0)
assert(round(n.predict([1,0,0,1])[0]) == 0)
assert(round(n.predict([1,0,1,0])[0]) == 0)
assert(round(n.predict([1,0,1,1])[0]) == 0)
assert(round(n.predict([1,1,0,0])[0]) == 0)
assert(round(n.predict([1,1,0,1])[0]) == 0)
assert(round(n.predict([1,1,1,0])[0]) == 0)
assert(round(n.predict([1,1,1,1])[0]) == 1)

for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                print("[", i, j, k, l, "]", round(n.predict([i, j, k, l])[0]))

In [ ]:
#Maybe the deeper the number of layers the more training that is needed
#Weights are not constrained to be positive or even -1.0 to 1.0 they can be any number